##1. Mounting data 

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


##2. Importing Relevant Libraries and Defining Training Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path = '/gdrive/My Drive/ML1 Course1 Sept 2020/Assignments/Data/'

In [ ]:
train_data = pd.read_csv(path+'mnist_train.csv', header=None)
train_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,147,252,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##3. Seperating X and Y And Converting them to Numpy arrays

In [ ]:
columns=train_data.shape[1]
print("Number of columns = {}".format(columns))

Number of columns = 785


In [ ]:
y = train_data.iloc[:,:1]
x = train_data.iloc[:,1:columns]

x=np.array(x)
y=np.array(y)

In [ ]:
x.shape, y.shape

((60000, 784), (60000, 1))

##4. Defining classes

In [ ]:
unique = np.unique(y)  #finding all unique classes in the output
print(unique)

[0 1 2 3 4 5 6 7 8 9]


In [ ]:
classes = unique.size
print(classes)

10


##5. Inserting Bias Term (training data)

In [ ]:
x.shape

(60000, 784)

In [ ]:
rows = x.shape[0]          #Finding the number of Rows (60,000)
x = np.insert(x, 0, values = np.ones(rows), axis=1)  #inserting ones into x

x.shape

(60000, 785)

##6. Defining all_theta with value 0

In [ ]:
params = x.shape[1]     #Finding the number of features
all_theta = np.zeros((classes, params)) 

all_theta.shape

(10, 785)

##7. Defining Sigmoid Function, Computing Cost and Defining Gradient

In [ ]:
def sigmoid(z):    #returns values between zero to one
    return 1 / (1 + np.exp(-z))

In [ ]:
def computeCost(theta, x, y):
    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)
    
    #(-y*log(g(z))) --> when expected output (y)=1
    first = np.multiply(-y, np.log(sigmoid(x * theta.T))) 
    
    #((1-y)*(1-log(g(z)))) --> when expected output (y)=0
    second = np.multiply((1 - y), np.log(1 - sigmoid(x * theta.T)))
    
    cost = np.sum(first - second)
    cost = cost/(2*len(x)) 
                 
    return cost

In [ ]:
cost= computeCost(all_theta, x, y)
print("cost = {}".format(cost))

cost = 3.4657359027997328


In [ ]:
def gradient(theta, x, y):
    
    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)
    learning_rate = 1
    
    error = sigmoid(x * theta.T) - y
    #grad. = (c/m) * (transpose(X) * ( sigmoid(X * transpose(theta))) )
    grad = learning_rate*((x.T * error) / len(x))
    
    return grad

##8. Modelling

In [ ]:
from scipy.optimize import minimize
def one_vs_all(x, y, classes, new_theta):
    params = x.shape[1] # number of columns in X array
    rows = x.shape[0] # number of rows in X array
      
    for i in range(0, classes):
        theta = new_theta[i,:]
        
        y_i = np.array([1 if label == i else 0 for label in y]) 
        y_i = np.reshape(y_i, (rows, 1))
                
        # minimize the objective function  --> Gradient descent
        fmin = minimize(fun=computeCost, x0=theta, args=(x, y_i), method='TNC', jac=gradient)
        all_theta[i,:] = fmin.x
        
        print(fmin.fun)
    
    return all_theta 

In [ ]:
new_theta = one_vs_all(x,y, classes, all_theta)

0.009116295132612584


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys


0.009115103658074138
0.0303278243302122


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in multiply
  # Remove the CWD from sys.path while we load stuff.


0.0411216099781866
0.020769321331743272
0.03791746507965672
0.01615060455308738
0.020516444914752404
0.054188586828285625
0.046090190118580045


In [ ]:
print('new_theta = {}'.format(new_theta))

new_theta = [[-5.06368776  0.          0.         ...  0.          0.
   0.        ]
 [-0.58587613  0.          0.         ...  0.          0.
   0.        ]
 [-2.99177414  0.          0.         ...  0.          0.
   0.        ]
 ...
 [-0.10498818  0.          0.         ...  0.          0.
   0.        ]
 [-7.35851283  0.          0.         ...  0.          0.
   0.        ]
 [-4.36195887  0.          0.         ...  0.          0.
   0.        ]]


In [ ]:
print("new cost = {}".format(computeCost(new_theta, x, y)))

new cost = 589.1259791502882


##9. Predicting for Training set

In [ ]:
def predict_all(x, theta): 
   
    # convert to matrices
    x = np.matrix(x)
    theta = np.matrix(theta)
    
    # run the model on each training instance
    h = sigmoid(x * theta.T)
    
    # return the column index that has the maximum confidence
    h_argmax = np.argmax(h, axis=1)
    
    return h_argmax, h

In [ ]:
y_pred, h = predict_all(x, new_theta)
print(y_pred)

[[5]
 [0]
 [4]
 ...
 [5]
 [6]
 [8]]


##10. Accuracy of Model for Training Data Prediction

In [ ]:
def acc(y_pred,y):
  correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
  accuracy = (sum(correct)) / (len(correct))
  return accuracy

In [ ]:
accuracy = acc(y_pred,y)

print ("Training data accuracy = {}%".format(accuracy * 100))

Training data accuracy = 93.11166666666666%


##11. Defining Test Data

In [ ]:
test_data = pd.read_csv(path+'mnist_test.csv', header=None)
test_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
columns=train_data.shape[1]
print("Number of columns = {}".format(columns))

Number of columns = 785


##12. Seperating X and Y for Test Data

In [ ]:
y = test_data.iloc[:,:1]
x = test_data.iloc[:,1:columns]

x=np.array(x)
y=np.array(y)

In [ ]:
x.shape, y.shape

((10000, 784), (10000, 1))

##13. Inserting Bias (for Test Data)

In [ ]:
t_rows = x.shape[0] #Finding the number of Rows (data samples)
t_x = np.insert(x, 0, values=np.ones(t_rows), axis=1) #inserting ones into x

##14. Prediction for Test Data

In [ ]:
t_y_pred, h = predict_all(t_x, new_theta)

##15. Accuracy of Model for Test Data Prediction

In [ ]:
print ("Test data accuracy = {}%".format(acc(t_y_pred, y) * 100))

Test data accuracy = 91.7%
